### 将python方程转化为tensor operation
tf.py_func
tf.py_func(
    func,
    inp,
    Tout,
    stateful=True,
    name=None
)
Defined in tensorflow/python/ops/script_ops.py.

Wraps a python function and uses it as a TensorFlow op.

Given a python function func, which takes numpy arrays as its arguments and returns numpy arrays as its outputs, wrap this function as an operation in a TensorFlow graph. The following snippet constructs a simple TensorFlow graph that invokes the np.sinh() NumPy function as a operation in the graph:

def my_func(x):
  ###### x will be a numpy array with the contents of the placeholder below
  return np.sinh(x)
input = tf.placeholder(tf.float32)
y = tf.py_func(my_func, [input], tf.float32)
N.B. The tf.py_func() operation has the following known limitations:

The body of the function (i.e. func) will not be serialized in a GraphDef. Therefore, you should not use this function if you need to serialize your model and restore it in a different environment.

The operation must run in the same address space as the Python program that calls tf.py_func(). If you are using distributed TensorFlow, you must run a tf.train.Server in the same process as the program that calls tf.py_func() and you must pin the created operation to a device in that server (e.g. using with tf.device():).

Args:
func: A Python function, which accepts ndarray objects as arguments and returns a list of ndarray objects (or a single ndarray). This function must accept as many arguments as there are tensors in inp, and these argument types will match the corresponding tf.Tensor objects in inp. The returns ndarrays must match the number and types defined Tout. Important Note: Input and output numpy ndarrays of func are not guaranteed to be copies. In some cases their underlying memory will be shared with the corresponding TensorFlow tensors. In-place modification or storing func input or return values in python datastructures without explicit (np.)copy can have non-deterministic consequences.
inp: A list of Tensor objects.
Tout: A list or tuple of tensorflow data types or a single tensorflow data type if there is only one, indicating what func returns.
stateful: (Boolean.) If True, the function should be considered stateful. If a function is stateless, when given the same input it will return the same output and have no observable side effects. Optimizations such as common subexpression elimination are only performed on stateless operations.
name: A name for the operation (optional).
Returns:
A list of Tensor or a single Tensor which func computes.



In [5]:
import tensorflow as tf
import numpy as np


### Demo1: 

In [9]:
def my_func(x):
    return np.sinh(x)
inputs = tf.placeholder(tf.float32)
y = tf.py_func(my_func, [inputs], tf.float32)

a = tf.constant(3.0, dtype=tf.float32)
x = 3.0
with tf.Session() as sess:
    #print (sess.run(y))  #Error, no inputs
    print (sess.run(y, feed_dict={inputs: x}))
    sess.close()

10.0179


### Demo2: 

In [57]:
def my_func2(x):
    length = len(x[0])
    #print (length)
    flags = np.empty((1, 5), dtype=np.bool)
    for i in range(0, length):
        #print (x[0][i])
        if x[0][i] >= 0:
            flags[0][i] = True
        else:
            flags[0][i] = False

    #new_array = np.array
    #for i in (0, length):
    #    if x[0][i] >= 0:
    #        flags[0][i] = True
    #    else:
    #        flags[0][i] = False
    return flags
inputs2 = tf.placeholder(tf.float32, shape=[1, 5])
y2 = tf.py_func(my_func2, [inputs2], tf.bool)
x2 = [[1.0, 2.0, -3.0, 1.6, 0.0]]
#print (x2[0][1])
with tf.Session() as sess:
    output = sess.run(y2, feed_dict={inputs2: x2})
    print ("Final outputs", output)
    print ("Output type: ", type(output))
    print ("Final outputs: ", sess.run(y2, feed_dict={inputs2: x2}))
    sess.close()



Final outputs [[ True  True False  True  True]]
Output type:  <class 'numpy.ndarray'>
Final outputs:  [[ True  True False  True  True]]


In [58]:
array1 = np.array([1.0, 2.0, 3.0, 4.0, 5.0])
new_array = np.array([False, False, False, False, False])
print (array1.shape)
for i in range(0, array1.shape[0]):
    if array1[i] > 0:
        new_array[i] = True
    print (new_array)

print ("--------------------------------")
a = np.empty((5), dtype=np.bool)
print (a.shape)
print (a.shape[0])
print (a)

(5,)
[ True False False False False]
[ True  True False False False]
[ True  True  True False False]
[ True  True  True  True False]
[ True  True  True  True  True]
--------------------------------
(5,)
5
[ True  True  True  True  True]


In [61]:
def my_func3(x1, x2):
    length = len(x1[0])
    flags = np.empty((1, 5), dtype=np.bool)
    for i in range(0, length):
        sum = x1[0][i] + x2[0][i]
        if sum > 0:
            flags[0][i] = True
        else:
            flags[0][i] = False
    return flags
inputs31 = tf.placeholder(tf.float32, shape=[1, 5])
inputs32 = tf.placeholder(tf.float32, shape=[1, 5])
y3 = tf.py_func(my_func3, [inputs31, inputs32], tf.bool)
x31 = [[1.0, 2.0, 3.0, 4.0, 5.0]]
x32 = [[0.0, -3.0, -4.0, 1.0, -4.0]]
with tf.Session() as sess:
    output = sess.run(y3, feed_dict={inputs31: x31, inputs32: x32})
    print ("Outputs3: ", output)
    sess.close()

Outputs3:  [[False False False False False]]
